In [ ]:
# default_exp environment

# Environment

> Environment/Callback

In [ ]:
#hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
# export

from mrl.imports import *
from mrl.torch_imports import *
from mrl.torch_core import *
from mrl.chem import *
from mrl.templates import *
from mrl.agent import *

//anaconda3/envs/mrl/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterCatalogEntry const> already registered; second conversion method ignored.
  return f(*args, **kwds)


In [ ]:
# export

class Callback():
    def __init__(self, order=1000):
        self.order=order
    
    def __call__(self, event_name):
        
        event = getattr(self, event_name, None)
        if event is not None:
            output = event()
        else:
            output = None
            
        return output
    
class BatchStats(Callback):
    def __init__(self):
        super().__init__(order=0)
        
        self.iterations = 0
        self.diversity = []
        self.valid = []
        self.rewards = []
        
    def before_train(self):
        print('\t'.join([key for key in list(self.__dict__.keys())]))
        
    def after_batch(self):
        outputs = []
        for k,v in self.__dict__.items():
            if type(v) == list:
                val = v[-1]
            else:
                val = v
            outputs.append(f'{val:.2f}')
        print('\t'.join(outputs))
        self.iterations += 1
    
class Buffer(Callback):
    def __init__(self, p_total, max_size=1000000):
        super().__init__(order=0)
        
        self.buffer = []
        self.used_buffer = []
        self.max_size = max_size
        self.p_total = p_total
        
    def __len__(self):
        return len(self.buffer)
    
    def add(self, item):
        
        if is_container(item):
            for i in item:
                self.add(i)
        else:
            place_idx = (len(self.buffer)%self.max_size)+1

            if place_idx>=len(self.buffer):
                self.buffer.append(item)
            else:
                self.buffer[place_idx-1] = item
            
    def sample(self, n):
        
        idxs = np.random.choice(np.arange(len(self.buffer)), n, replace=False)
        batch = [self.buffer[i] for i in idxs]
        for idx in sorted(idxs, reverse=True):
            self.buffer.pop(idx)
            
        self.used_buffer += batch
        self.used_buffer = self.used_buffer[-self.max_size:]
        
        return batch
    
    def after_build_buffer(self):
        self.buffer = self.environment.template_cb.filter_sequences(self.buffer)
    
    def sample_batch(self):
        bs = self.environment.bs * self.p_total
        sample = self.sample(bs)
        self.batch_state.samples.append(sample)
        self.batch_state.sources.append(['buffer']*len(sample))

In [ ]:
# export

class SettrDict(dict):
    def __init__(self):
        super().__init__()
        
    def __setitem__(self, key, item):
        super().__setitem__(key, item)
        super().__setattr__(key, item)
    
    def __setattr__(self, key, item):
        super().__setitem__(key, item)
        super().__setattr__(key, item)
        
    def update_from_dict(self, update_dict):
        for k,v in update_dict.items():
            self[k] = v
        
class BatchState(SettrDict):
    def __init__(self):
        super().__init__()
        
        self.samples = []
        self.sources = []
        self.rewards = to_device(torch.tensor(0.))
        self.rewards_scaled = to_device(torch.tensor(0.))
        self.trajectory_rewards = to_device(torch.tensor(0.))
        self.loss = to_device(torch.tensor(0.))
        
#         self.sequence_trajectories = []
#         self.x = None
#         self.y = None
#         self.mask = None
#         self.sl = None
#         self.model_output = None
#         self.model_encoded = None
#         self.model_logprobs = None
#         self.model_gathered_logprobs = None
#         self.y_gumbel = None
#         self.vhead_values = None
#         self.old_vhead_values = None
#         self.ref_output = None
#         self.ref_encoded = None
#         self.ref_logprobs = None
#         self.ref_gathered_logprobs = None
#         self.trajectory_rewards = None


In [ ]:
# export

class Event():
    def __init__(self):
        self.setup = 'setup'
        self.before_train = 'before_train'
        self.build_buffer = 'build_buffer'
        self.after_build_buffer = 'after_build_buffer'
        self.before_batch = 'before_batch'
        self.sample_batch = 'sample_batch'
        self.after_sample = 'after_sample'
        self.get_model_outputs = 'get_model_outputs'
        self.compute_reward = 'compute_reward'
        self.after_compute_reward = 'after_compute_reward'
        self.compute_loss = 'compute_loss'
        self.zero_grad = 'zero_grad'
        self.step = 'step'
        self.after_batch = 'after_batch'
        self.after_train = 'after_train'

In [ ]:
# export

class Environment():
    def __init__(self, agent_cb, template=None, reward_cbs=[], loss_cbs=[], cbs=[]):
        self.agent_cb = agent_cb
        self.template_cb = TemplateCallback(template)
        self.reward_cbs = reward_cbs
        self.loss_cbs = loss_cbs
        self.cbs = []
        self.register_cbs([agent_cb]+[template_cb]+reward_cbs+loss_cbs+cbs)
        self.buffer = Buffer()
        self.batch_state = BatchState()
        self.batch_stats = BatchStats()
        self.mean_reward = None
        self('setup')
        
    def __call__(self, event):
        for cb in self.cbs:
            if hasattr(cb, event):
                cb(event)
        
    def register_cb(self, cb):
        if isinstance(cb, type): 
            cb = cb()
        cb.environment = self
        setattr(self, cb.name, cb)
        sel.cbs.append(cbs)
        
    def register_cbs(self, cbs):
        for cb in cbs:
            self.register_cb(cb)
            
    def build_bufer(self):
        if len(self.buffer) < self.bs:
            self('build_buffer')
            self('after_build_buffer')
            
    def sample_batch(self):
        self.batch_state = BatchState()
        for cb in self.cbs:
            cb.batch_state = self.batch_state
        self('before_batch') 
        self('sample_batch') 
        self.batch_stats.diversity.append(
            len(set(self.batch_stats.sequences))/len(self.batch_stats.sequences))
        self.batch_stats.valid.append(
            len([i for i in sequences if to_mol(i) is not None])/len(sequences))
        self('after_sample') 
        
    def compute_rewards(self):
        self('compute_rewards')
        rewards = self.batch_state.rewards
        
        if self.mean_reward is None:
            self.mean_reward = rewards.mean()
        else:
            self.mean_reward = (1-self.reward_decay)*rewards.mean() + self.reward_decay*self.mean_reward
            
        rewards_scaled = rewards - self.mean_reward
        self.batch_state.rewards_scaled = rewards_scaled
        self('after_compute_rewards')
        
    def compute_loss(self):
        self('compute_loss')
        loss = self.batch_state.loss
        self('zero_grad')
        loss.backward()
        self('step')
            
    def fit(self, bs, sl, iters, buffer_size):
        self.bs = bs
        self.sl = sl
        self.buffer_size = buffer_size
        self('before_train')
        for step in range(iters):
            self.build_buffer()
            self.sample_batch()
            self.agg_batch()
            self('get_model_outputs')
            self.compute_rewards()
            self('after_batch')
        self('after_train')
        

In [ ]:
class Sampler(Callback):
    def __init__(self, name, p_buffer=0., p_batch=0.):
        super().__init__()
        self.name = name
        self.p_buffer = p_buffer
        self.p_batch = p_batch
        
    def setup(self):
        if self.p_batch>0.:
            bs = self.environment.batch_stats
            setattr(bs, f'{self.name}_diversity', [])
            setattr(bs, f'{self.name}_valid', [])
            setattr(bs, f'{self.name}_rewards', [])
            setattr(bs, f'{self.name}_new', [])
        
    def build_buffer(self):
        pass
    
    def sample_batch(self):
        pass
    
    def after_compute_reward(self):
        if self.p_batch>0:
            state = self.environment.batch_state
            stats = self.environment.batch_stats
            rewards = state.rewards.detach().cpu().numpy()
            sources = np.array(state.sources)
            if self.name in sources:
                getattr(stats, f'{self.name}_rewards').append(rewards[sources==self.name].mean())
            else:
                getattr(stats, f'{self.name}_rewards').append(0.)
                
            
    
class ModelSampler(Sampler):
    def __init__(self, agent, model, name, p_buffer, p_batch, latent=False):
        super().__init__(name, p_buffer, p_batch)
        self.agent = agent
        self.model = model
        self.latent = latent if self.agent.latents is not None else False
        
    def build_buffer(self):
        env = self.environment
        bs = int(env.bs * self.p_buffer)
        if bs > 0:
            

In [ ]:
class TemplateCallback(Callback):
    def __init__(self, template=None):
        super().__init__(order=-1)
        self.template = template
        
    def compute_reward(self):
        env = self.environment
        state = 
        
    def filter_sequences(self, sequences):
        if self.templaate is not None:
            hp = np.array(self.template(sequences))
            sequences = np.array(sequences)[hp]
            sequences = list(sequences)
            
        return sequences

In [ ]:
# export

class Event():
    def __init__(self):
        self.setup = 'setup'
        self.before_train = 'before_train'
        self.build_buffer = 'build_buffer'
        self.after_build_buffer = 'after_build_buffer'
        self.before_batch = 'before_batch'
        self.sample_batch = 'sample_batch'
        self.after_sample = 'after_sample'
        self.get_model_outputs = 'get_model_outputs'
        self.compute_reward = 'compute_reward'
        self.after_compute_reward = 'after_compute_reward'
        self.compute_loss = 'compute_loss'
        self.zero_grad = 'zero_grad'
        self.step = 'step'
        self.after_batch = 'after_batch'
        self.after_train = 'after_train'

In [ ]:
bs = BatchStats()

In [ ]:
setattr(bs, 'asad', [])

In [ ]:
bs.asad

[]

In [ ]:
getattr(bs, 'asad').append(5)

In [ ]:
bs.asad

[5]